# Домашнее задание 3

## Сбор данных из интернета и генерация текстов

В этом домашнем задании вам предстоит: 
1. выбрать свой источник структурированных данных (прогноз погоды, курс акций, счет спортивных мероприятий, или что-нибудь еще) 
2. научиться извлекать данные из с выбранной веб-страницы
3. научиться генерировать текст по шаблонам

## Часть 1

Выберете источник данных, который вам симпатичен. По умолчанию, предлагаем использовать данные о погоде в Москве: 
https://www.gismeteo.ru/weather-moscow-4368/. Извлеките из html-кода страницы температуру сегодня, уровень осадков и скорость ветра.

## Часть 2
Запрограммируйте шаблон, описывающие температуру, уровень осадков и скорость ветра сегодня. Например,
"Температура сегодня составляет X градусов", где Х – излеченное выше число. Сгенерируйте таким образом описание погоды сегодня. Учтите, величина X влияет на грамматическое число, использованное в единицах измерения ("двадцать градусов", но "двадцать один градус"). Используйте бибилиотеку utg или дополнительные функции pymorphy2

In [1]:
import re
from nltk import tokenize
from nltk.corpus import stopwords
from pymystem3 import Mystem


In [2]:
import pandas as pd
import urllib.request

from scrapy.selector import Selector
from scrapy.http import HtmlResponse
from tqdm import tqdm


In [3]:
url = 'https://www.gismeteo.ru/weather-moscow-4368/'
req = urllib.request.Request(url, headers={'User-Agent': "Magic Browser"})
html = urllib.request.urlopen(url)
html = html.read()

скачаем температуру

In [493]:
tempr = Selector(text = html).xpath('/html/body/section/div/div/div[1]/div[3]/div[1]/div[1]/div/div/div[1]/div[3]/div/div/div/div[1]/text()').extract()[0]
print(tempr)

+1


скачаем атмосферное давление

In [350]:
pres = Selector(text = html).xpath('/html/body/section/div/div/div[1]/div[3]/div[9]/div/div[2]/div[2]/div[2]/div/div/div/div[6]/text()').extract()[0]
print(pres)

747


скачаем скорость и направление ветра

In [216]:
wind_speed = Selector(text = html).xpath('/html/body/section/div/div/div[1]/div[3]/div[7]/div/div[2]/div[2]/div[2]/div[7]/div/div[1]/span/text()').extract()[0]
print(wind_speed)

6


In [330]:
direction_wind = Selector(text = html).xpath('/html/body/section/div/div/div[1]/div[3]/div[7]/div/div[2]/div[2]/div[2]/div[6]/div/div[3]/text()').extract()[0]
direction_wind = ' '.join(direction_wind.split())
print(direction_wind)

ЮВ


скачаем влажность воздуха

In [376]:
humidity =  Selector(text = html).xpath('/html/body/section/div/div/div[1]/div[3]/div[10]/div/div[2]/div[2]/div[2]/div[4]/div/text()').extract()[0]
print(humidity)

90


сделаем шаблоны погоды с помощью utg и pymorphy2

In [279]:
from utg import relations as r
from utg import logic
from utg import data
from utg import dictionary
from utg import words
from utg import templates
from utg import constructors
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [494]:
def generate_forms(word):
    p = morph.parse(word)[0]
    res = []
    lexems = p.lexeme
    if 'NOUN' in p.tag:
        for i in range(len(lexems)):
            res.append(lexems[i][4][0][1])
        #print(res)
        if 'femn' in p.tag:
            res1 = res[:5]
            res1.append(res[6])
            res1.extend(res[7:])
            res1.extend(res1[6:])
            return(res1)
        else:
            res.extend(res[6:])
            return(res)
    if 'VERB' in p.tag or 'INFN' in p.tag:
        for i in [0,1,3,2,4]:
            res.append(lexems[i][4][0][1])
        return(res)
    if 'ADJF' in p.tag:
        for i in range(18):
            res.append(lexems[i][4][0][1])
        return(res)

In [495]:
noun_word = words.Word(type=r.WORD_TYPE.NOUN,
                        forms=generate_forms('градус'),
                        properties=words.Properties(r.ANIMALITY.ANIMATE, r.GENDER.MASCULINE))
test_dictionary = dictionary.Dictionary(words=[noun_word])

In [496]:
template = templates.Template()

In [497]:

temp = words.WordForm(words.Word(type=r.WORD_TYPE.NOUN,
                                forms=generate_forms('температура'),
                                 properties=words.Properties(r.ANIMALITY.ANIMATE, r.GENDER.FEMININE)))
vozd = words.WordForm(words.Word(type=r.WORD_TYPE.NOUN,
                                 forms=generate_forms('воздух')[:18] ,
                                 properties=words.Properties(r.ANIMALITY.INANIMATE, r.GENDER.MASCULINE)))

In [498]:
template.parse(u'[temp] [vozd|рд] сегодня + [coins] [градус|coins|вн].', externals=( 'temp', 'vozd', 'coins'))


In [499]:
result = template.substitute(externals={'temp': temp,
                                        'vozd' : vozd,
                                        'coins': constructors.construct_integer(int(tempr))},
                             dictionary=test_dictionary)

In [500]:
result

'температура воздуха сегодня + 1 градус.'

In [483]:
def veter(direction_wind):
    if direction_wind == 'ЮВ':
        return 'юго-восточный'
    if direction_wind == 'ЮЗ':
        return 'юго-западный'
    if direction_wind == 'СВ':
        return 'северо-восточный'
    if direction_wind == 'СЗ':
        return 'северо-западный'
    if direction_wind == 'Ю':
        return 'южный'
    if direction_wind == 'З':
        return 'западный'
    if direction_wind == 'С':
        return 'северный'
    if direction_wind == 'В':
        return 'восточный'
veter(direction_wind)

'юго-восточный'

In [484]:
metr = morph.parse('метр')[0] 
sec = morph.parse('секунда')[0] 
print('Ветер', veter(direction_wind),',', int(wind_speed), metr.make_agree_with_number(int(wind_speed)).word ,'в', sec.inflect({'accs'}).word ,'.') 

Ветер юго-восточный , 6 метров в секунду .


In [491]:
atm = morph.parse('атмосферное')[0] 
pr = morph.parse('давление')[0] 
mm = morph.parse('миллиметр')[0] 
rt = morph.parse('ртутный')[0] 
st = morph.parse('столб')[0] 
print(atm.inflect({'nomn'}).word, pr.inflect({'nomn'}).word,int(pres), mm.make_agree_with_number(int(pres)).word, rt.inflect({'gent'}).word, st.inflect({'gent'}).word, '.' )

атмосферное давление 747 миллиметров ртутного столба .


In [486]:
noun_word = words.Word(type=r.WORD_TYPE.NOUN,
                        forms=generate_forms('влага'),
                        properties=words.Properties(r.ANIMALITY.ANIMATE, r.GENDER.FEMININE))
test_dictionary = dictionary.Dictionary(words=[noun_word])
template = templates.Template()
otn = words.WordForm(words.Word(type=r.WORD_TYPE.ADJECTIVE,
                                forms=generate_forms('относительная')[5:] + [u''] * (5+len(data.WORDS_CACHES[r.WORD_TYPE.ADJECTIVE]) - len(generate_forms('относительная'))),
                                 properties=words.Properties(r.ANIMALITY.ANIMATE, r.GENDER.FEMININE, r.CASE.NOMINATIVE)))
vozd = words.WordForm(words.Word(type=r.WORD_TYPE.NOUN,
                                 forms=generate_forms('воздух')[:18] ,
                                 properties=words.Properties(r.ANIMALITY.INANIMATE, r.GENDER.MASCULINE)))
percent = words.WordForm(words.Word(type=r.WORD_TYPE.NOUN,
                                 forms=generate_forms('процент')[:18] ,
                                 properties=words.Properties(r.ANIMALITY.INANIMATE, r.GENDER.MASCULINE)))

template.parse(u' [otn|им|жр] [влага] [vozd|рд] [coins] [percent|coins|вн].', externals=( 'otn', 'vozd', 'percent', 'coins'))

result1= template.substitute(externals={'otn': otn,
                                        'vozd' : vozd,
                                        'percent': percent,
                                        'coins': constructors.construct_integer(int(humidity))},
                             dictionary=test_dictionary)

In [487]:
result1

' относительная влага воздуха 90 процентов.'

In [501]:
print(result)
print(result1)
print('Ветер', veter(direction_wind),',', int(wind_speed), metr.make_agree_with_number(int(wind_speed)).word ,'в', sec.inflect({'accs'}).word ,'.')
print(atm.inflect({'nomn'}).word, pr.inflect({'nomn'}).word,int(pres), mm.make_agree_with_number(int(pres)).word, rt.inflect({'gent'}).word, st.inflect({'gent'}).word, '.' )

температура воздуха сегодня + 1 градус.
 относительная влага воздуха 90 процентов.
Ветер юго-восточный , 6 метров в секунду .
атмосферное давление 747 миллиметров ртутного столба .
